In [1]:
import os
import sys

spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()

import pyspark

In [2]:
#these need to be tweaked for workers
number_cores = 8
memory_gb = 24
conf = (pyspark.SparkConf().setMaster('spark://192.168.1.1:7077'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [3]:
!ls -lh ../data/yelp/

total 8617536
-rw-r--r--  1 lngo  staff    25M Nov  9 18:02 yelp_academic_dataset_business.json.gz
-rw-r--r--  1 lngo  staff   2.3G Nov  9 18:02 yelp_academic_dataset_review.json.gz
-rw-r--r--  1 lngo  staff   1.8G Oct 21 15:00 yelp_academic_dataset_user.json.gz


### Identify 100 users with highest number of ratings/fans.  

- User dataset
- You can decide/justify this ranking and decide the importance of ratings/fans. 
- Focusing on fans is better. 


#### Step 1: 
- Load the data:
  - We can use SQL: SQlContext

In [5]:
# import SQLContext and create a sqlContext
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [6]:
# get SQL table
df_json = sqlContext.read.json("../data/yelp/yelp_academic_dataset_user.json.gz")

In [ ]:
df_json.printSchema()

In [8]:
df_json.registerTempTable("tbl_json")

  - PySpark way

In [14]:
df_data = sc.textFile("../data/yelp/yelp_academic_dataset_user.json.gz")
print(df_data.count())
df_data.take(1)

1968703


['{"user_id":"ntlvfPzc8eglqvk92iDIAw","name":"Rafael","review_count":553,"yelping_since":"2007-07-06 03:27:11","useful":628,"funny":225,"cool":227,"elite":"","friends":"oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfDg, IkRib6Xs91PPW7pon7VVig, A8Aq8f0-XvLBcyMk2GJdJQ, eEZM1kogR7eL4GOBZyPvBA, e1o1LN7ez5ckCpQeAab4iw, _HrJVzFaRFUhPva8cwBjpQ, pZeGZGzX-ROT_D5lam5uNg, 0S6EI51ej5J7dgYz3-O0lA, woDt8raW-AorxQM_tIE2eA, hWUnSE5gKXNe7bDc8uAG9A, c_3LDSO2RHwZ94_Q6j_O7w, -uv1wDiaplY6eXXS0VwQiA, QFjqxXn3acDC7hckFGUKMg, ErOqapICmHPTN8YobZIcfQ, mJLRvqLOKhqEdkgt9iEaCQ, VKX7jlScJSA-ja5hYRw12Q, ijIC9w5PRcj3dWVlanjZeg, CIZGlEw-Bp0rmkP8M6yQ9Q, OC6fT5WZ8EU7tEVJ3bzPBQ, UZSDGTDpycDzrlfUlyw2dQ, deL6e_z9xqZTIODKqnvRXQ, 5mG2ENw2PylIWElqHSMGqg, Uh5Kug2fvDd51RYmsNZkGg, 4dI4uoShugD9z84fYupelQ, EQpFHqGT9Tk6YSwORTtwpg, o4EGL2-ICGmRJzJ3GxB-vw, s8gK7sdVzJcYKcPv2dkZXw, vOYVZgb_GVe-kdtjQwSUHw, wBbjgHsrKr7BsPBrQwJf2w, p59u2EC_qcmCmLeX1jCi5Q, VSAZI1eHDrOPRWMK4Q2DIQ, efMfeI_dkhpeGykaRJqxfQ, x6qYcQ8_i0mMDzSLsFCbZg, K_zSmtNGw1fu-vm

#### Step 2:
  - Get the information using SQL Statement

In [9]:
df_json.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [12]:
# 100 users with highest number of ratings/fans.
highest_fan = sqlContext.sql("SELECT user_id, review_count, fans FROM tbl_json ORDER BY fans DESC LIMIT 100")

In [13]:
highest_fan.show()

+--------------------+------------+-----+
|             user_id|review_count| fans|
+--------------------+------------+-----+
|37cpUoM8hlkSQfReI...|        1787|11568|
|hizGc5W1tBHPghM5Y...|        1629| 3315|
|eKUGKQRE-Ywi5dY55...|        2430| 2916|
|Hi10sGSZNxQH3NLyW...|       11112| 2718|
|j14WgRoU_-2ZE1aw1...|        3566| 2634|
|iLjMdZi0Tm7DQxX1C...|        2431| 2516|
|JjXuiru1_ONzDkYVr...|        1207| 2316|
|ITa3vh5ERI90G_WP4...|        3079| 2280|
|UsXqCXRZwSCSw0AT7...|        2919| 2263|
|VHdY6oG2JPVNjihWh...|        1979| 2140|
|fgwI3rYHOv1ipfVfC...|        1847| 2113|
|nkN_do3fJ9xekchVC...|        1363| 2046|
|m07sy7eLtOjVdZ8oN...|        4913| 2034|
|peuxbSQwXed-81cSq...|        1592| 1916|
|AHRrG3T1gJpHvtpZ-...|        2109| 1747|
|WG3w_73scm_JUWJ_3...|        5013| 1733|
|NfU0zDaTMEQ4-X9db...|        1137| 1726|
|lt7bNHl-TXziny4FE...|         802| 1708|
|3zxy3LVBV3ttxoYbY...|        1894| 1696|
|ysCBsXWPB-LAiewVS...|        3687| 1672|
+--------------------+------------

  - Get the information using PySpark Statement

In [15]:
df_data.take(1)

['{"user_id":"ntlvfPzc8eglqvk92iDIAw","name":"Rafael","review_count":553,"yelping_since":"2007-07-06 03:27:11","useful":628,"funny":225,"cool":227,"elite":"","friends":"oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfDg, IkRib6Xs91PPW7pon7VVig, A8Aq8f0-XvLBcyMk2GJdJQ, eEZM1kogR7eL4GOBZyPvBA, e1o1LN7ez5ckCpQeAab4iw, _HrJVzFaRFUhPva8cwBjpQ, pZeGZGzX-ROT_D5lam5uNg, 0S6EI51ej5J7dgYz3-O0lA, woDt8raW-AorxQM_tIE2eA, hWUnSE5gKXNe7bDc8uAG9A, c_3LDSO2RHwZ94_Q6j_O7w, -uv1wDiaplY6eXXS0VwQiA, QFjqxXn3acDC7hckFGUKMg, ErOqapICmHPTN8YobZIcfQ, mJLRvqLOKhqEdkgt9iEaCQ, VKX7jlScJSA-ja5hYRw12Q, ijIC9w5PRcj3dWVlanjZeg, CIZGlEw-Bp0rmkP8M6yQ9Q, OC6fT5WZ8EU7tEVJ3bzPBQ, UZSDGTDpycDzrlfUlyw2dQ, deL6e_z9xqZTIODKqnvRXQ, 5mG2ENw2PylIWElqHSMGqg, Uh5Kug2fvDd51RYmsNZkGg, 4dI4uoShugD9z84fYupelQ, EQpFHqGT9Tk6YSwORTtwpg, o4EGL2-ICGmRJzJ3GxB-vw, s8gK7sdVzJcYKcPv2dkZXw, vOYVZgb_GVe-kdtjQwSUHw, wBbjgHsrKr7BsPBrQwJf2w, p59u2EC_qcmCmLeX1jCi5Q, VSAZI1eHDrOPRWMK4Q2DIQ, efMfeI_dkhpeGykaRJqxfQ, x6qYcQ8_i0mMDzSLsFCbZg, K_zSmtNGw1fu-vm

In [17]:
import json

df_top_users = df_data.map(lambda x: json.loads(x)).map(lambda x: (x["user_id"], x["review_count"], x["fans"]))
df_top_users.take(1)

[('ntlvfPzc8eglqvk92iDIAw', 553, 14)]

In [20]:
def extract_user(x):
    x_json = json.loads(x)
    return (x_json["user_id"], x_json["review_count"], x_json["fans"])

tmp = df_data.take(1)
extract_user(tmp[0])

('ntlvfPzc8eglqvk92iDIAw', 553, 14)

In [22]:
df_top_users = df_data.map(extract_user).takeOrdered(100, lambda x: -x[2])
df_top_users

[('37cpUoM8hlkSQfReIEBd-Q', 1787, 11568),
 ('hizGc5W1tBHPghM5YKCAtg', 1629, 3315),
 ('eKUGKQRE-Ywi5dY55_zChg', 2430, 2916),
 ('Hi10sGSZNxQH3NLyWSZ1oA', 11112, 2718),
 ('j14WgRoU_-2ZE1aw1dXrJg', 3566, 2634),
 ('iLjMdZi0Tm7DQxX1C1_2dg', 2431, 2516),
 ('JjXuiru1_ONzDkYVrHN0aw', 1207, 2316),
 ('ITa3vh5ERI90G_WP4SmGUQ', 3079, 2280),
 ('UsXqCXRZwSCSw0AT7y1uBg', 2919, 2263),
 ('VHdY6oG2JPVNjihWhOooAQ', 1979, 2140),
 ('fgwI3rYHOv1ipfVfCSx7pg', 1847, 2113),
 ('nkN_do3fJ9xekchVC-v68A', 1363, 2046),
 ('m07sy7eLtOjVdZ8oN9JKag', 4913, 2034),
 ('peuxbSQwXed-81cSqL7Ykw', 1592, 1916),
 ('AHRrG3T1gJpHvtpZ-K0G_g', 2109, 1747),
 ('WG3w_73scm_JUWJ_3Lgn0Q', 5013, 1733),
 ('NfU0zDaTMEQ4-X9dbQWd9A', 1137, 1726),
 ('lt7bNHl-TXziny4FETu8nA', 802, 1708),
 ('3zxy3LVBV3ttxoYbY4rQ8A', 1894, 1696),
 ('ysCBsXWPB-LAiewVS3jZfQ', 3687, 1672),
 ('wEE-YMx5pmSuagLtNxMPKA', 1845, 1653),
 ('djxnI8Ux8ZYQJhiOQkrRhA', 2315, 1614),
 ('G9Vb6yQ047TC3O_-GG4WZA', 1540, 1574),
 ('JADdo9NEeO5Az9aOYbyvZA', 2382, 1464),
 ('58yXn5Y4409k

In [24]:
highest_fan.show()

+--------------------+------------+-----+
|             user_id|review_count| fans|
+--------------------+------------+-----+
|37cpUoM8hlkSQfReI...|        1787|11568|
|hizGc5W1tBHPghM5Y...|        1629| 3315|
|eKUGKQRE-Ywi5dY55...|        2430| 2916|
|Hi10sGSZNxQH3NLyW...|       11112| 2718|
|j14WgRoU_-2ZE1aw1...|        3566| 2634|
|iLjMdZi0Tm7DQxX1C...|        2431| 2516|
|JjXuiru1_ONzDkYVr...|        1207| 2316|
|ITa3vh5ERI90G_WP4...|        3079| 2280|
|UsXqCXRZwSCSw0AT7...|        2919| 2263|
|VHdY6oG2JPVNjihWh...|        1979| 2140|
|fgwI3rYHOv1ipfVfC...|        1847| 2113|
|nkN_do3fJ9xekchVC...|        1363| 2046|
|m07sy7eLtOjVdZ8oN...|        4913| 2034|
|peuxbSQwXed-81cSq...|        1592| 1916|
|AHRrG3T1gJpHvtpZ-...|        2109| 1747|
|WG3w_73scm_JUWJ_3...|        5013| 1733|
|NfU0zDaTMEQ4-X9db...|        1137| 1726|
|lt7bNHl-TXziny4FE...|         802| 1708|
|3zxy3LVBV3ttxoYbY...|        1894| 1696|
|ysCBsXWPB-LAiewVS...|        3687| 1672|
+--------------------+------------

### Extract the reviews of these users and combine it with the business information. 

- Are they continental, regional, or local eaters? 
    - Look at review data for business data, look at business data for location. 
    - A dataset exists, is posted on Discord. 
    - Second options: distance between furthest pair of restaurants, cluster into three. 
- Is there a preference in restaurant/food style of their reviews? 
    - How/Can we phrase this as a frequent items question?
        - per user
            - set of items: categories
            - set of baskets: restaurants
- Can you infer the locations of these users?
    - pay attention to timing patterns of review

### Identify one of your favorite restaurants that is available on Yelp. Search for all reviews and reviewers for this restaurants. 

- Is this restaurant frequented by non-local reviewers (how do you know)?
- What are the positive things about this restaurant (study higher-rated reviews)
- What are the negative things about this restaurant (study lower-rated reviews)